In [2]:
import pandas as pd
import json

In [13]:
def load_Data(user, filename):
   data = json.load(open('dir.json'))
   dir = data[user] + filename
   data = pd.read_csv(dir)
   return data

In [18]:
data = load_Data('Yash', 'AG01-02.csv')
data.head()

                                                                                                                                                                         ;"mean.vm";"sd.vm";"mean.ang";"sd.ang";"p625";"dfreq";"ratio.df";"start.time";"METs.rf";"METs.lm";"MET.lev.rf";"MET.lev.tr";"sed.rf";"sed.tr";"loc.rf";"loc.tr";"combo.rf";"combo.tr";"day";"hour";"session";"sed.truth"
7805;1 05326856910668;0 194844517038988;-27 7176053044508;26 8078263893347;0 180772708206207;2 80467445742905;0  012283642402803;2017-10-03 18:46:00;2  87764822045946;3  71665016260102;"light";"light";"non-sedentary"...                                                                                                                                                                      
7806;1 06648604153266;0 275645377906665;-50 1408942707846;27 6646250708782;0 246832951353768;0 734557595993322;0 01481874229078;2017-10-03 18:46:15;4   08997277603327;4  76816142693718;"moderate";"moderate";"non-sede...         

,,,,,,,,,";""mean.vm"";""sd.vm"";""mean.ang"";""sd.ang"";""p625"";""dfreq"";""ratio.df"";""start.time"";""METs.rf"";""METs.lm"";""MET.lev.rf"";""MET.lev.tr"";""sed.rf"";""sed.tr"";""loc.rf"";""loc.tr"";""combo.rf"";""combo.tr"";""day"";""hour"";""session"";""sed.truth"""
7805;1,05326856910668;0,194844517038988;-27,7176053044508;26,8078263893347;0,180772708206207;2,80467445742905;0,012283642402803;2017-10-03 18:46:00;2,87764822045946;3,"71665016260102;""light"";""light"";""non-sedentary""..."
7806;1,06648604153266;0,275645377906665;-50,1408942707846;27,6646250708782;0,246832951353768;0,734557595993322;0,01481874229078;2017-10-03 18:46:15;4,08997277603327;4,"76816142693718;""moderate"";""moderate"";""non-sede..."
7807;1,10941255288145;0,308551262478439;-41,5516616232119;27,6980382826044;0,184373690414555;1,3355592654424;0,0132545539145475;2017-10-03 18:46:30;4,43835099047722;4,"71711615273235;""moderate"";""moderate"";""non-sede..."
7808;1,10426328299936;0,3866312335354;-45,1973655833661;25,0146683098872;0,167807254696256;0,534223706176962;0,0134813002239392;2017-10-03 18:46:45;4,15147512897604;5,"24579598881731;""moderate"";""moderate"";""non-sede..."
7809;1,09443370314476;0,30474907342321;-22,8691709148691;26,6874724843314;0,151519880284707;0,200333889816361;0,0179313219674924;2017-10-03 18:47:00;3,87234514577937;4,"1909005482331;""moderate"";""moderate"";""non-seden..."
